In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
re = pd.read_pickle('recommend_cat_new.pkl')
re.head()

,isbn,title,categoryName,salesPoint,adult,rank,genres,mood,interest
198,9791165341909,달러구트 꿈 백화점 (50만 부 기념 드림 에디션) - 주문하신 꿈은 매진입니다,소설,379054,False,8,"[역사, 동화, 로맨스, 미스터리, 판타지, 무협]","[행복, 위로]","[일기, 동물, 사랑, 생각, 진로, 설득]"
559,9791161571188,불편한 편의점 (40만부 기념 벚꽃 에디션),소설,330837,False,10,"[시대, 동화]","[위로, 힐링, 행복]","[시간, 심리, 관계, 사랑, 생각]"
40339,9791192186313,시맨틱 에러 포토에세이,에세이,242520,False,10,[판타지],[위로],"[사랑, 청춘, 영화, 사진]"
199,9788954681179,밝은 밤 (별밤 에디션),소설,204554,False,10,"[예술, 동화, 로맨스]","[행복, 위로, 열정]","[사랑, 생각, 인생, 청춘, 관계, 시간, 가족]"
200,9791165343729,달러구트 꿈 백화점 2 - 단골손님을 찾습니다,소설,191493,False,9,"[동화, 로맨스, 미스터리, 판타지, 무협]","[행복, 위로]","[사랑, 생각, 인생]"


In [3]:
re = re.reset_index()

In [32]:
카테고리 = []
장르 = []
감성 = []
관심사 = []
성인 = []
장르no = []
감성no = []
관심사no = []
성인no = []

#book = pd.read_pickle('recommend.pkl')
성인질문 = input('성인입니까? : 네 / 아니요\n')
if 성인질문 == '네':
    성인.append([True,False])
    카테고리.append(input("\n원하시는 분야를 입력해주세요 : \n소설 / 에세이 / 자기계발 / 시 / 희곡 / 선택없음\n").split())
    장르.append(input('\n책을 읽을 때 선호하는 장르가 있나요? : \n시대 / 역사 / 철학 / 로맨스 / 전쟁 /  미스터리 / 예술 /  판타지 / 자서전 / 동화 / 과학 / 무협 / 선택없음\n').split())
    장르no.append(input('원하지 않는 장르를 입력해주세요 : \n').split())
    감성.append(input('\n어떤 감성의 책을 원하세요? : \n고민 / 열정 / 도전 / 긍정 / 후회 / 행복 / 우울 / 위로 / 고통 / 자유 /\
                    \n희망 / 슬픔 / 열망 / 비극 / 불안 / 용기 / 절망 / 불행 / 유머 / 힐링 / 선택없음\n').split())
    감성no.append(input('원하지 않는 감성를 입력해주세요 : \n').split())
    관심사.append(input('\n요즘 관심있는 것이 무엇인가요? : \n리더십 / 시간 / 페미니즘 / 취업 / 진로 / 비즈니스 / 설득 / 건강 / 심리 / 시 / 희곡 / 가족 / 관계 / 동물 / 사진 / 그림 / 여행 / 미술 / 영화 / 음악 / 음식 / 일기 / 자연 / 종교 / 사랑 / 인간 / 생각 / 사회 / 인생 / 일상 / 친구 / 성공 / 청춘 / 정치 / 취미 / 스포츠 / 선택없음\n').split())
    관심사no.append(input('원하지 않는 관심사를 입력해주세요 : \n').split())
elif 성인질문 == '아니요':
    성인.append([False])
    카테고리.append(input("\n원하시는 분야를 입력해주세요 : \n소설 / 에세이 / 자기계발 / 시 / 희곡 / 선택없음\n").split())
    장르.append(input('\n책을 읽을 때 선호하는 장르가 있나요? : \n시대 / 역사 / 철학 / 로맨스 / 전쟁 /  미스터리 / 예술 /  판타지 / 자서전 / 동화 / 과학 / 무협 / 선택없음\n').split())
    장르no.append(input('원하지 않는 장르를 입력해주세요 : \n').split())
    감성.append(input('\n어떤 감성의 책을 원하세요? : \n고민 / 열정 / 도전 / 긍정 / 후회 / 행복 / 우울 / 위로 / 고통 / 자유 /\
                    \n희망 / 슬픔 / 열망 / 비극 / 불안 / 용기 / 절망 / 불행 / 유머 / 힐링 / 선택없음\n').split())
    감성no.append(input('원하지 않는 감성를 입력해주세요 : \n').split())
    관심사.append(input('\n요즘 관심있는 것이 무엇인가요? : \n리더십 / 시간 / 페미니즘 / 취업 / 진로 / 비즈니스 / 설득 / 건강 / 심리 / 시 / 희곡 / 가족 / 관계 / 동물 / 사진 / 그림 / 여행 / 미술 / 영화 / 음악 / 음식 / 일기 / 자연 / 종교 / 사랑 / 인간 / 생각 / 사회 / 인생 / 일상 / 친구 / 성공 / 청춘 / 정치 / 취미 / 스포츠 / 선택없음\n').split())
    관심사no.append(input('원하지 않는 관심사를 입력해주세요 : \n').split())

성인입니까? : 네 / 아니요
네

원하시는 분야를 입력해주세요 : 
소설 / 에세이 / 자기계발 / 시 / 희곡 / 선택없음
소설 에세이

책을 읽을 때 선호하는 장르가 있나요? : 
시대 / 역사 / 철학 / 로맨스 / 전쟁 /  미스터리 / 예술 /  판타지 / 자서전 / 동화 / 과학 / 무협 / 선택없음
역사 전쟁
원하지 않는 장르를 입력해주세요 : 
자서전

어떤 감성의 책을 원하세요? : 
고민 / 열정 / 도전 / 긍정 / 후회 / 행복 / 우울 / 위로 / 고통 / 자유 /                    
희망 / 슬픔 / 열망 / 비극 / 불안 / 용기 / 절망 / 불행 / 유머 / 힐링 / 선택없음
선택없음
원하지 않는 감성를 입력해주세요 : 
후회 우울

요즘 관심있는 것이 무엇인가요? : 
리더십 / 시간 / 페미니즘 / 취업 / 진로 / 비즈니스 / 설득 / 건강 / 심리 / 시 / 희곡 / 가족 / 관계 / 동물 / 사진 / 그림 / 여행 / 미술 / 영화 / 음악 / 음식 / 일기 / 자연 / 종교 / 사랑 / 인간 / 생각 / 사회 / 인생 / 일상 / 친구 / 성공 / 청춘 / 정치 / 취미 / 스포츠 / 선택없음
청춘 리더십
원하지 않는 관심사를 입력해주세요 : 
선택없음


In [33]:
성인, 카테고리, 장르, 장르no, 감성, 감성no, 관심사, 관심사no

([[True, False]],
 [['소설', '에세이']],
 [['역사', '전쟁']],
 [['자서전']],
 [['선택없음']],
 [['후회', '우울']],
 [['청춘', '리더십']],
 [['선택없음']])

In [17]:
# 관심있는 것
def category(카테고리):
    if 카테고리[0][0] != '선택없음':
        return (re['categoryName'][i] in 카테고리[0])
    else:
        return True

def genresand(장르):
    if 장르[0][0] == '선택없음':
        return True
    elif (len(장르[0]) == 1):
        return (장르[0][0] in re['genres'][i])
    elif (len(장르[0]) == 2):
        return (장르[0][0] in re['genres'][i]) & (장르[0][1] in re['genres'][i])


def moodand(감성):
    if 감성[0][0] == '선택없음':
        return True
    elif (len(감성[0]) == 1):
        return (감성[0][0] in re['mood'][i])
    elif (len(감성[0]) == 2):
        return (감성[0][0] in re['mood'][i]) & (감성[0][1] in re['mood'][i])

def interestand(관심사):
    if 관심사[0][0] == '선택없음':
        return True
    elif (len(관심사[0]) == 1):
        return (관심사[0][0] in re['interest'][i])
    elif (len(관심사[0]) == 2):
        return (관심사[0][0] in re['interest'][i]) & (관심사[0][1] in re['interest'][i])
    
def genresor(장르):
    if 장르[0][0] == '선택없음':
        return True
    elif (len(장르[0]) == 1):
        return (장르[0][0] in re['genres'][i])
    elif (len(장르[0]) == 2):
        return (장르[0][0] in re['genres'][i]) | (장르[0][1] in re['genres'][i])


def moodor(감성):
    if 감성[0][0] == '선택없음':
        return True
    elif (len(감성[0]) == 1):
        return (감성[0][0] in re['mood'][i])
    elif (len(감성[0]) == 2):
        return (감성[0][0] in re['mood'][i]) | (감성[0][1] in re['mood'][i])

def interestor(관심사):
    if 관심사[0][0] == '선택없음':
        return True
    elif (len(관심사[0]) == 1):
        return (관심사[0][0] in re['interest'][i])
    elif (len(관심사[0]) == 2):
        return (관심사[0][0] in re['interest'][i]) | (관심사[0][1] in re['interest'][i])

In [29]:
# 관심없는 것

def nogenresand(장르no):
    if 장르no[0][0] == '선택없음':
        return True
    elif (len(장르no[0]) == 1):
        return (장르no[0][0] not in re['genres'][i])
    elif (len(장르no[0]) == 2):
        return (장르no[0][0] not in re['genres'][i]) & (장르no[0][1] not in re['genres'][i])


def nomoodand(감성no):
    if 감성no[0][0] == '선택없음':
        return True
    elif (len(감성no[0]) == 1):
        return (감성no[0][0] not in re['mood'][i])
    elif (len(감성no[0]) == 2):
        return (감성no[0][0] not in re['mood'][i]) & (감성no[0][1] not in re['mood'][i])

def nointerestand(관심사no):
    if 관심사no[0][0] == '선택없음':
        return True
    elif (len(관심사no[0]) == 1):
        return (관심사no[0][0] not in re['interest'][i])
    elif (len(관심사no[0]) == 2):
        return (관심사no[0][0] not in re['interest'][i]) & (관심사no[0][1] not in re['interest'][i])

In [24]:
관심사[0][0] == '선택없음', interest(관심사)

(True, True)

In [210]:
 카테고리[0]

('소설', ['에세이'])

In [216]:
(re['adult'][0] in 성인[0])

True

In [34]:
select_list_and = []
for i in range(len(re)):
    if (re['adult'][i] in 성인[0]) & category(카테고리) & genresand(장르) & moodand(감성) & interestand(관심사)\
    & nogenresand(장르no) & nomoodand(감성no) & nointerestand(관심사no) :
        select_list_and.append(re.loc[i])
        
select_book_and = pd.DataFrame(select_list_and)

select_list_or = []
for i in range(len(re)):
    if (re['adult'][i] in 성인[0]) & category(카테고리) & genresor(장르) & moodor(감성) & interestor(관심사)\
    & nogenresand(장르no) & nomoodand(감성no) & nointerestand(관심사no):
        select_list_or.append(re.loc[i])
        
select_book_or = pd.DataFrame(select_list_or)

if len(select_book_and) >= 20:
    a = select_book_and
else:
    a = select_book_or

a

## score

,index,isbn,title,categoryName,salesPoint,adult,rank,genres,mood,interest
28,1196,9788936433673,엄마를 부탁해,소설,83218,False,9,"[역사, 시대, 동화]","[위로, 열정, 행복]","[사랑, 청춘, 가족, 시간, 심리, 인생, 사회, 인간]"
30,220,9788937460050,동물농장,소설,81901,False,9,"[예술, 시대, 역사, 동화]","[자유, 유머]","[정치, 동물, 스포츠, 사회, 인간, 리더십]"
53,228,9788937437564,참을 수 없는 존재의 가벼움 - 국내 출간 30주년 기념 특별판,소설,57571,False,9,"[예술, 시대, 역사]","[행복, 열정]","[생각, 사랑, 인생, 청춘, 인간]"
67,43012,9788971844427,지도 밖으로 행군하라,에세이,48830,False,9,[역사],"[자유, 열망]","[일기, 리더십]"
118,716,9788937460753,위대한 개츠비,소설,35787,False,9,"[전쟁, 시대, 미스터리]","[열정, 행복, 도전, 유머]","[심리, 사랑, 인생, 청춘, 성공]"
...,...,...,...,...,...,...,...,...,...,...
53846,38868,9791165399542,불사조 1 : 선과 악,소설,20,True,0,"[동화, 전쟁]","[위로, 고통, 희망, 슬픔, 불안]","[페미니즘, 청춘, 생각, 인생, 인간]"
53914,38822,9791190526500,계변쌍학무,소설,20,False,0,"[역사, 동화, 철학, 로맨스, 전쟁]","[열정, 행복]","[사랑, 인생, 청춘, 종교]"
54079,39326,9788952241214,[큰글자] 레 미제라블 2,소설,20,False,0,"[예술, 시대, 역사, 철학]","[희망, 행복, 위로, 열정]","[사랑, 생각, 인생, 청춘, 사회, 종교]"
54080,39325,9788952241207,[큰글자] 레 미제라블 1,소설,20,False,0,"[예술, 시대, 역사, 철학]","[희망, 행복, 위로, 열정]","[사랑, 생각, 인생, 청춘, 사회, 종교]"


In [8]:
len(select_book)

27

In [25]:
성인[0]

[True, False]

In [26]:
re['adult'][0]

False